In [1]:
# 1. Prepare the Environment

pip install pandas numpy faker openpyxl xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 11.1 MB/s eta 0:00:00


In [2]:
# 2. Import Necessary Modules

import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime
import random
import uuid


In [3]:
# 3. Define Utility Functions

def generate_uuid():
    """Generate a random UUID."""
    return str(uuid.uuid4())

def generate_random_date(start_date, end_date):
    """Generate a random date between start_date and end_date."""
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + pd.Timedelta(days=random_days)


In [4]:
# 4. Set Up Faker and Field Types

fake = Faker()

FIELD_TYPES = {
    "Name": lambda: fake.name(),
    "Full Name": lambda: fake.name(),
    "Address": lambda: fake.address(),
    "Phone": lambda: fake.phone_number(),
    "Email": lambda: fake.email(),
    "UUID": generate_uuid,
    "Company": lambda: fake.company(),
    "Department": lambda: fake.job(),
    "City": lambda: fake.city(),
    "Country": lambda: fake.country(),
    "Zip Code": lambda: fake.zipcode(),
    "Product Name": lambda: fake.catch_phrase(),
    "State or Province": lambda: fake.state(),
}


In [14]:
# 5. Define the Masking Function

def mask_data(df, selected_columns, config_settings, keep_mapping=True):
    masked_df = df.copy()  # Copy the original data
    log = []

    # Dictionary to store mappings for each column if "Keep Mapping Consistent" is enabled
    column_fake_mappings = {}

    # Iterate over selected columns and apply fake data where needed
    for col, settings in selected_columns.items():
        if settings["selected"]:
            field_type = settings["field_type"]
            blank_percent = settings.get("blank_percent", 0.0)

            # Generate fake data with or without keeping mappings
            if keep_mapping:
                # Generate mappings for unique values in the column to keep them consistent
                unique_values = df[col].unique()
                if col not in column_fake_mappings:
                    fake_mapping = {}
                    if field_type == "Custom List":
                        values = config_settings[col].get("values", [])
                        for idx, val in enumerate(unique_values):
                            fake_mapping[val] = (
                                None
                                if np.random.rand() < blank_percent
                                else values[idx % len(values)]
                            )
                    elif field_type == "Number":
                        min_val = config_settings[col].get("min", 0)
                        max_val = config_settings[col].get("max", 1)
                        is_integer = config_settings[col].get("is_integer", False)
                        for val in unique_values:
                            if np.random.rand() < blank_percent:
                                fake_value = None
                            else:
                                if is_integer:
                                    fake_value = random.randint(int(min_val), int(max_val))
                                else:
                                    fake_value = round(random.uniform(min_val, max_val), 2)
                            fake_mapping[val] = fake_value
                    elif field_type == "Date":
                        start_date = datetime.strptime(
                            config_settings[col].get("start_date", "2020-01-01"),
                            "%Y-%m-%d",
                        )
                        end_date = datetime.strptime(
                            config_settings[col].get("end_date", "2023-12-31"),
                            "%Y-%m-%d",
                        )
                        for val in unique_values:
                            fake_mapping[val] = (
                                None
                                if np.random.rand() < blank_percent
                                else generate_random_date(start_date, end_date)
                            )
                    elif field_type == "UUID":
                        prefix = config_settings[col].get("prefix", "")
                        suffix = config_settings[col].get("suffix", "")
                        uuid_type = config_settings[col].get("uuid_type", "UUID")
                        char_length = config_settings[col].get("char_length", 8)
                        for val in unique_values:
                            if np.random.rand() < blank_percent:
                                fake_value = None
                            else:
                                if uuid_type == "UUID":
                                    fake_value = f"{prefix}{generate_uuid()}{suffix}"
                                elif uuid_type == "Alphanumeric Code":
                                    code = fake.bothify('?' * char_length)
                                    fake_value = f"{prefix}{code}{suffix}"
                                else:
                                    raise ValueError(f"Unknown uuid_type: {uuid_type}")
                            fake_mapping[val] = fake_value
                    else:
                        for val in unique_values:
                            fake_mapping[val] = (
                                None
                                if np.random.rand() < blank_percent
                                else FIELD_TYPES[field_type]()
                            )
                    column_fake_mappings[col] = fake_mapping
                # Map values based on the generated mapping
                masked_df[col] = df[col].map(column_fake_mappings[col])
            else:
                # Generate new fake data for each row independently when keep_mapping is False
                if field_type == "Custom List":
                    values = config_settings[col].get("values", [])
                    fake_data = [
                        None if np.random.rand() < blank_percent else random.choice(values)
                        for _ in range(len(df))
                    ]
                elif field_type == "Number":
                    min_val = config_settings[col].get("min", 0)
                    max_val = config_settings[col].get("max", 1)
                    is_integer = config_settings[col].get("is_integer", False)
                    fake_data = []
                    for _ in range(len(df)):
                        if np.random.rand() < blank_percent:
                            fake_value = None
                        else:
                            if is_integer:
                                fake_value = random.randint(int(min_val), int(max_val))
                            else:
                                fake_value = round(random.uniform(min_val, max_val), 2)
                        fake_data.append(fake_value)
                elif field_type == "Date":
                    start_date = datetime.strptime(
                        config_settings[col].get("start_date", "2020-01-01"), "%Y-%m-%d"
                    )
                    end_date = datetime.strptime(
                        config_settings[col].get("end_date", "2023-12-31"), "%Y-%m-%d"
                    )
                    fake_data = [
                        None if np.random.rand() < blank_percent else generate_random_date(start_date, end_date)
                        for _ in range(len(df))
                    ]
                elif field_type == "UUID":
                    prefix = config_settings[col].get("prefix", "")
                    suffix = config_settings[col].get("suffix", "")
                    uuid_type = config_settings[col].get("uuid_type", "UUID")
                    char_length = config_settings[col].get("char_length", 8)
                    fake_data = []
                    for _ in range(len(df)):
                        if np.random.rand() < blank_percent:
                            fake_value = None
                        else:
                            if uuid_type == "UUID":
                                fake_value = f"{prefix}{generate_uuid()}{suffix}"
                            elif uuid_type == "Alphanumeric Code":
                                code = fake.bothify('?' * char_length)
                                fake_value = f"{prefix}{code}{suffix}"
                            else:
                                raise ValueError(f"Unknown uuid_type: {uuid_type}")
                        fake_data.append(fake_value)
                else:
                    fake_data = [
                        None if np.random.rand() < blank_percent else FIELD_TYPES[field_type]()
                        for _ in range(len(df))
                    ]
                # Apply generated fake data to the entire column
                masked_df[col] = fake_data
            log.append(
                f"Masked column '{col}' with fake data ({field_type}) and {blank_percent*100}% blanks."
            )
        else:
            log.append(f"Column '{col}' was not selected for masking.")

    return masked_df, log

In [9]:
# 6. Load Your Data

# For CSV files
# df = pd.read_csv('path_to_your_file.csv')

# For Excel files
df = pd.read_excel('/content/Sample - Superstore.xls')

df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2020-103800,2020-01-03,2020-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2020-141817,2020-01-05,2020-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


# **Test Cases and Examples**

This section provides test cases and examples for using the mask_data function in your Jupyter notebook.

### **Test Case 1: Masking with Custom List**

```
selected_columns = {
    'Category': {
        'selected': True,
        'field_type': 'Custom List',
        'blank_percent': 0.1  # 10% blanks
    },
}

config_settings = {
    'Category': {
        'values': ['Option A', 'Option B', 'Option C']
    },
}
```

### **Test Case 2: Masking with Number**

```
selected_columns = {
    'Age': {
        'selected': True,
        'field_type': 'Number',
        'blank_percent': 0.05  # 5% blanks
    },
}

config_settings = {
    'Age': {
        'min': 18,
        'max': 65,
        'is_integer': True
    },
}
```

### **Test Case 3: Masking with Date**

```
selected_columns = {
    'SignupDate': {
        'selected': True,
        'field_type': 'Date',
        'blank_percent': 0.0
    },
}

config_settings = {
    'SignupDate': {
        'start_date': '2020-01-01',
        'end_date': '2023-12-31'
    },
}
```

### **Test Case 4: Masking with UUID (UUID Type)**

```
selected_columns = {
    'UserID': {
        'selected': True,
        'field_type': 'UUID',
        'blank_percent': 0.0
    },
}

config_settings = {
    'UserID': {
        'prefix': 'USER-',
        'suffix': '',
        'uuid_type': 'UUID',  # Standard UUID
    },
}
```

### **Test Case 5: Masking with UUID (Alphanumeric Code)**

```
selected_columns = {
    'UserID': {
        'selected': True,
        'field_type': 'UUID',
        'blank_percent': 0.0
    },
}

config_settings = {
    'UserID': {
        'prefix': 'ID-',
        'suffix': '-END',
        'uuid_type': 'Alphanumeric Code',
        'char_length': 10  # Length of the alphanumeric code
    },
}
```

### **Test Case 6: Masking with Blanks Introduced**

```
selected_columns = {
    'Email': {
        'selected': True,
        'field_type': 'Email',
        'blank_percent': 0.2  # 20% blanks
    },
}

config_settings = {
    # No additional settings needed for 'Email' field type
}
```

### **Test Case 7: Combined Example - Masking Multiple Columns**

```
selected_columns = {
    'Name': {
        'selected': True,
        'field_type': 'Name',
        'blank_percent': 0.0
    },
    'Email': {
        'selected': True,
        'field_type': 'Email',
        'blank_percent': 0.1  # 10% blanks
    },
    'Age': {
        'selected': True,
        'field_type': 'Number',
        'blank_percent': 0.0
    },
    'SignupDate': {
        'selected': True,
        'field_type': 'Date',
        'blank_percent': 0.0
    },
    'UserID': {
        'selected': True,
        'field_type': 'UUID',
        'blank_percent': 0.0
    },
    'Category': {
        'selected': True,
        'field_type': 'Custom List',
        'blank_percent': 0.0
    },
}

config_settings = {
    'Age': {
        'min': 18,
        'max': 65,
        'is_integer': True
    },
    'SignupDate': {
        'start_date': '2020-01-01',
        'end_date': '2023-12-31'
    },
    'UserID': {
        'prefix': 'USER-',
        'suffix': '',
        'uuid_type': 'UUID',
    },
    'Category': {
        'values': ['Option A', 'Option B', 'Option C']
    },
}
```

# Notes
*   **Adjusting keep_mapping**: Set keep_mapping to False if you want each occurrence to be masked independently, resulting in different fake data even for identical original values.
*   **Customizing Field Types**: You can add more field types or adjust existing ones in the FIELD_TYPES dictionary.
*   **Error Handling**: Ensure that the configurations are correctly specified to avoid errors during masking.


In [15]:
# 7. Define Your Masking Configuration

selected_columns = {
    'Segment': {
        'selected': True,
        'field_type': 'Custom List',
        'blank_percent': 0.0
    },
    'Customer ID': {
        'selected': True,
        'field_type': 'UUID',
        'blank_percent': 0.0
    }
}

config_settings = {
    'Segment': {
        'values': ['Seg1', 'Seg2', 'Seg3', 'Seg4'],
    },
    'Customer ID': {
        'prefix': 'START-',
        'suffix': '-END',
        'uuid_type': 'Alphanumeric Code',
        'char_length': 10  # Length of the alphanumeric code
    }
}


In [16]:
# 8. Run the Masking Function

masked_df, log = mask_data(df, selected_columns, config_settings, keep_mapping=True)


In [17]:
# 9. Review the Logs

for entry in log:
    print(entry)


Masked column 'Segment' with fake data (Custom List) and 0.0% blanks.
Masked column 'Customer ID' with fake data (UUID) and 0.0% blanks.


In [ ]:
# 10. Save or Use the Masked Data

# Save to Excel
masked_df.to_excel('masked_data.xlsx', index=False)

# Save to CSV
# masked_df.to_csv('masked_data.csv', index=False)


In [18]:
# 11. (Optional) Display the Masked Data

masked_df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2020-103800,2020-01-03,2020-01-07,Standard Class,START-XpJKLfTFKZ-END,Darren Powers,Seg1,United States,Houston,...,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2020-112326,2020-01-04,2020-01-08,Standard Class,START-QqmJMfjLFc-END,Phillina Ober,Seg2,United States,Naperville,...,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2020-112326,2020-01-04,2020-01-08,Standard Class,START-QqmJMfjLFc-END,Phillina Ober,Seg2,United States,Naperville,...,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2020-112326,2020-01-04,2020-01-08,Standard Class,START-QqmJMfjLFc-END,Phillina Ober,Seg2,United States,Naperville,...,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2020-141817,2020-01-05,2020-01-12,Standard Class,START-wevRTiWjaN-END,Mick Brown,Seg1,United States,Philadelphia,...,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840
